In [1]:
import numpy as np
import pandas as pd

# データ等の準備

In [5]:
train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis = 1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

train_xは学習データ、train_yは目的変数、test_xはテストデータ\
pandasのDataFrame, Seriesで保持。(numpyのarrayで保持することもある)

In [6]:
from sklearn.model_selection import KFold

In [9]:
kf = KFold(n_splits = 4, shuffle = True, random_state = 71)
tr_idx, va_idx = list(kf.split(train_x))[0]

In [11]:
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

学習データを学習データとバリデーションデータに分ける

xgboostにおけるカスタム評価指標と目的関数の例\
(参考)https://github.com/dmlc/xgboost/blob/master/demo/guide-python/custom_objective.py

In [12]:
import xgboost as xgb
from sklearn.metrics import log_loss

dtrain = xgb.DMatrix(tr_x, label = tr_y)
dvalid = xgb.DMatrix(va_x, label = va_y)

特徴量と目的変数をxgboostのデータ構造に変換する。\
学習データの特徴量と目的変数がtr_x, tr_y, バリデーションデータの特徴量と目的変数がva_x, va_yとする。

In [17]:
def logregobj(preds, dtrain):
    labels = dtrain.get_label() #真の値のラベルを取得
    preds = 1.0 / (1.0 + np.exp(-preds)) #シグモイド関数
    grad = preds - labels  #勾配
    hess = preds * (1.0 - preds) #二階微分値
    return grad, hess

カスタム目的関数(この場合はloglossであり、xgboostの'binary:logistic'と等価)

In [19]:
def evaluerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'custom-error', float(sum(labels != (preds > 0.0))) / len(labels)

カスタム評価指標(この場合は誤答率)

In [ ]:
p